# Deployment of Sentinel-2 RGB Macrolocalization Model - 2020

This notebook deploys the Sentinel-2 RGB macrolocalization models for cement and steel plants for the year 2020.

Deployement plan is as follows:

1. Loop over each grid id.
2. Select all scenes between May - August 2020.
3. Choose the scene with least amount of cloud coverage, and score.
4. Iterate over scenes ordered by cloud coverage, to fill in full deployment region intersecting with grid id.

This idea is to reduce the amount of compute time required by reducing the complexing of how to find the best chip, and score whole scenes at once.

## Import required libraries

In [ ]:
!pip install fastai==1.0.61

In [ ]:
from earthai.all import *
import earthai.chipping.strategy as chp
import pyspark.sql.functions as pys
from pyspark.sql.window import Window

import geopandas as gpd
import pandas as pd
import rasterio

from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import Polygon

import os
import shutil
import boto3
import glob
import time
import sys

from shapely.wkt import loads
import re

from fastai import *
from fastai.vision import *

from IPython.display import clear_output

## Create Spark Session

* Important to do this before defining the udfs for scoring
* Set number of partitions on par with the number of catalog items per scene

In [ ]:
partitions = 250
spark = create_earthai_spark_session(**{
    "spark.default.parallelism": partitions,
    "spark.sql.shuffle.partitions": partitions,
})

## Define input/output files and paths, and parameters

### Parameters

* `chip_size` is the size of chips (length) to create (in pixels)
* `year` defines the year of selected scenes; months restricted to May - August.
* `scene_subset` set to 1 or 2. This divides the scoring in two pieces to run on two servers at the same time. 1 will process the first set of scenes; 2 will process the second. 
* `calc_crs` is a physical CRS used to compute distance from chip center and known plants that intersect.
* `scale_size` slightly reduces the scene extent to avoid spending too much times scoring scene edges.

In [ ]:
chip_size = 300 # 3 km for Sentinel-2
year = '2020'
scene_subset = 3
calc_crs = "EPSG:3395"
scale_size = 0.95

### Input files and paths

* `s2_deployment_gjson` is the deployment region for the S2 RGB model; used to define region to score
* `s2_scene_gjson` is the GeoJSON defining Sentinel-2 grid-ids to query; used to loop over scenes for deployment process
* `s2_grid_gjson` is the 10km grid and prediction values from infrastructure density model; joined to results at the end to merge scores
* `cement_plant_gjson` gives exact lat/long of cement plants; joined to results at the end to find chips intersection with known plants
* `steel_plant_gjson` gives exact lat/long of steel plants; joined to results at the end to find chips intersection with known plants
* `CEMENT_MODEL_PATH` is the path on S3 to the VGG13 multiclass model (best recall for cement: 89.1%)
* `STEEL_MODEL_PATH` is the path on S3 to the DenseNet161 multiclass model (best recall for steel: 89.7%)
* `LOCAL_DIR` specifies where to put files locally for analysis

In [ ]:
s2_deployment_gjson = '../../resources/macro-loc-model-deployment4/S2-deployment-region-CHN-10km-nowater.geojson'
s2_scene_gjson = '../../resources/macro-loc-model-deployment4/S2-deployment-scene-extents-CHN-10km-nowater.geojson'

s2_grid_gjson = '../../resources/macro-loc-model-deployment4/S2-deployment-grid-CHN-10km-nowater.geojson'
cement_plant_gjson = '../../resources/macro-loc-model-build4/cement_exact_china_v4.1_s2.geojson'
steel_plant_gjson = '../../resources/macro-loc-model-build4/steel_exact_china_v4.1_s2.geojson'

CEMENT_MODEL_PATH = 'S2-RGB-macro-localization-model-build4/S2-RGB-model-results4/vgg13_multiclass_final.pkl'
STEEL_MODEL_PATH = 'S2-RGB-macro-localization-model-build4/S2-RGB-model-results4/densenet161_multiclass_final.pkl'
LOCAL_DIR = '/scratch/'

### Output files and paths

* `s3_path` defines S3 high-level folder for S2 RGB macro-localization deployment results
* `output_path` defines (temporary) local place of storage
* `output_score_tar` define output tar of score GeoJSONS (one for each scene)
* `output_gjson_prefix` is the prefix for output GeoJSON files with scores for each scene

In [ ]:
s3_path = 'S2-RGB-macro-localization-model-deployment4'

output_path = 'S2-deployment-chip-scores-CHN-10km-nowater-'+year+'-set'+str(scene_subset)
output_score_tar = output_path+'.tar'
output_gjson_prefix = 'S2-deployment-chip-scores-CHN-10km-nowater-'+year+'-'

In [ ]:
if not os.path.exists(output_path):
    os.mkdir(output_path)

## Download Model and Define Scoring Function

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('sfi-shared-assets')

### Download models and load learners

In [ ]:
def download_model(MODEL_PATH):
    if not os.path.exists(LOCAL_DIR+MODEL_PATH.split("/")[-1].replace(".pkl", "")):
        os.makedirs(LOCAL_DIR + MODEL_PATH.split("/")[-1].replace(".pkl", ""))
    bucket.download_file(MODEL_PATH, LOCAL_DIR+MODEL_PATH.split("/")[-1].replace(".pkl", "") + "/export.pkl")

In [ ]:
download_model(CEMENT_MODEL_PATH)
download_model(STEEL_MODEL_PATH)

In [ ]:
cement_model = load_learner(LOCAL_DIR+CEMENT_MODEL_PATH.split("/")[-1].replace(".pkl", ""))
steel_model = load_learner(LOCAL_DIR+STEEL_MODEL_PATH.split("/")[-1].replace(".pkl", ""))

### Define scoring function for PNGs

In [ ]:
def score_pngs(path):
    
    # Get ImageDataBunch for Fastai
    data = (ImageDataBunch.from_folder(path, train='all', bs=16, num_workers=0, seed=42).normalize(imagenet_stats))
    
    # Create empty lists to store results
    data_cnt = len(data.train_ds)
    scene_id = []
    tile_id = []
    cement_prob = []
    steel_prob = []
    
    # Loop over images and get scores and metadata
    for i in range(0, data_cnt):
        
        # Cement model probability
        p_cement_model = cement_model.predict(data.train_ds.x[i])
        cement_prob.append(to_np(p_cement_model[2])[0].item())
    
        # Steel model probability
        p_steel_model = steel_model.predict(data.train_ds.x[i])
        steel_prob.append(to_np(p_steel_model[2])[2].item())
    
        # Metadata for chip
        scene_id.append('-'.join(str(data.items[i]).split('/')[-1].split('-')[0:2]))
        tile_id.append(str(data.items[i]).split('/')[-1].split('.')[0])
        
    # Return data frame
    score_pdf = pd.DataFrame({'s2_grid_id': scene_id,
                              'tile_id': tile_id,
                              'tile_cmt_prob': cement_prob,
                              'tile_stl_prob': steel_prob})
    
    return(score_pdf)

## Define EOD Catalog Read and Chipping Functions

### Get catalog of Sentinel-2 scenes that intersect with deployment regions

Queries EarthAI Catalog to find S2 scenes that intersect with deployment region.

* Takes single grid_id; returns scenes/datetimes from May - August
* Join back to deployment region for later clipping

In [ ]:
def calc_scene_deployment_region(base_gdf, overlay_gdf):
    
    # Find interesection of the base GeoDataFrame and the Overlay GeoDataFrame
    intersect_gdf = gpd.overlay(base_gdf, overlay_gdf, how='intersection')
    
    # Convert Polygons to Multipolygons
    multipoly_geom = [MultiPolygon([geom]) if (geom.type == 'Polygon') else geom for geom in intersect_gdf.geometry]
    intersect_gdf = intersect_gdf.set_geometry(multipoly_geom)
    
    # Change precision on lat/long to 0.0001 (~10m at equator)
    simpledec = re.compile(r"\d*\.\d+")
    def mround(match):
        return "{:.4f}".format(float(match.group()))
    intersect_gdf.geometry = intersect_gdf.geometry.apply(lambda x: loads(re.sub(simpledec, mround, x.wkt)))
    
    return(intersect_gdf)

In [ ]:
def calc_region_difference(base_gdf, overlay_gdf):
    
    # Get simplied representation of scored region from chips extents
    chip_geoms_union = overlay_gdf.unary_union
    if isinstance(chip_geoms_union, Polygon) or isinstance(chip_geoms_union, MultiPolygon):
        chip_geoms_union = [chip_geoms_union]
        
    chip_geoms_union = [MultiPolygon([x]) if (x.type == 'Polygon') else x for x in chip_geoms_union]
    scored_region_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(chip_geoms_union),
                                         crs='EPSG:4326')
    
    # Compute and return remaining region to score
    difference_gdf = gpd.overlay(base_gdf, scored_region_gdf, how='difference')
    diff_geom = [MultiPolygon([x]) if (x.type == 'Polygon') else x for x in difference_gdf.geometry]
    difference_gdf.geometry = gpd.GeoSeries(diff_geom)
    
    return(difference_gdf)

In [ ]:
def eod_read_catalog(geom, grid_id, year, max_cc=100):
    
    # Start/end date formatting
    start_date = year+'-05-01'
    end_date = year+'-08-31'
    
    # Query catalog
    site_cat = earth_ondemand.read_catalog(
        geo=geom,
        start_datetime=start_date,
        end_datetime=end_date,
        max_cloud_cover=max_cc,
        collections='sentinel2_l2a',
        grid_ids=[grid_id]
        )
    if len(site_cat) > 0:
        
        # Sort by cloud coverage and return
        site_cat = site_cat.sort_values('eo_cloud_cover')
        return(site_cat)
        
    else:
        return([])

## Create Image Chips

* Read and create image chips for 10km grid
* Select highest quality scene

In [ ]:
def create_chips(site_cat, chip_size=300):
    
    # Uses scene aligned grid to create same-size chips
    # Grabs red, green, and blue bands
    # Filter out chips smaller than chip_size x chips_size
    # Rename columns
    # Normalize and convert data bands to uint16
    w = Window().orderBy(lit('dummy'))
    site_chip_filt = spark.read.chip(site_cat, ['B04_10m','B03_10m','B02_10m'],
                                    chipping_strategy=chp.SceneAlignedGrid(chip_size,chip_size)) \
                         .select('eod_grid_id', 'id', 'datetime', 'eo_cloud_cover', 
                                 'B04_10m', 'B03_10m', 'B02_10m') \
                         .withColumn('tile_dims', rf_dimensions('B04_10m')) \
                         .filter((pys.col('tile_dims').rows == chip_size) & 
                                 (pys.col('tile_dims').cols == chip_size)) \
                         .withColumn('Red_uint16', rf_convert_cell_type('B04_10m', 'uint16')) \
                         .withColumn('nodata_cell_cnt', rf_no_data_cells('Red_uint16')) \
                         .filter(pys.col('nodata_cell_cnt') == 0).cache()
    isNotEmptyRF = len(site_chip_filt.head(1))
    
    if isNotEmptyRF == 1:
        site_chip_unq = site_chip_filt.withColumnRenamed('eod_grid_id', 's2_grid_id') \
                         .withColumnRenamed('eo_cloud_cover', 's2_eo_cloud_cover') \
                         .withColumnRenamed('id', 's2_id') \
                         .withColumnRenamed('datetime', 's2_datetime') \
                         .withColumn('tile_id', pys.concat_ws('-',
                                                              pys.col('s2_grid_id'), 
                                                              pys.lpad(pys.row_number().over(w), 4, '0'))) \
                         .withColumn('Red', 
                                 rf_convert_cell_type(
                                     rf_local_multiply(
                                         rf_rescale(rf_convert_cell_type('B04_10m', 'uint16')), 65535), 'uint16')) \
                         .withColumn('Green', 
                                 rf_convert_cell_type(
                                     rf_local_multiply(
                                         rf_rescale(rf_convert_cell_type('B03_10m', 'uint16')), 65535), 'uint16')) \
                         .withColumn('Blue', 
                                 rf_convert_cell_type(
                                     rf_local_multiply(
                                         rf_rescale(rf_convert_cell_type('B02_10m', 'uint16')), 65535), 'uint16')) \
                         .drop('B04_10m', 'B03_10m', 'B02_10m', 'tile_dims', 'Red_uint16', 'nodata_cell_cnt') \
                         .cache()
    
        return(site_chip_unq)
    
    else:
        return(site_chip_filt)

## Convert GeoTIFFs to PNGs

In [ ]:
def convert_image(tif_filename, png_filename):
    with rasterio.open(tif_filename) as infile:
        
        profile = infile.profile
        profile['driver'] = 'PNG'
        
        raster = infile.read()
        
        with rasterio.open(png_filename, 'w', **profile) as dst:
            dst.write(raster)

## Create PNGs from RasterFrame

In [ ]:
def png_from_rf(rf):
    
    # Create GeoTIFFs from RasterFrame
    rf.write.chip('geotiffs', filenameCol='tile_id', catalog=False)
    tif_file_list = glob.glob('geotiffs/*.tif')
    
    # Create output paths for PNGs to fit Fastai structure
    os.mkdir('pngs')
    os.mkdir('pngs/all')
    png_file_list = [f.replace('.tif', '.png').replace('geotiffs/', 'pngs/all/') for f in tif_file_list]
    
    # Convert and write out PNGs
    for i in range(0, len(tif_file_list)):
        convert_image(tif_file_list[i], png_file_list[i])

## Define Output Function

* Writes out scores to GeoJSON file

In [ ]:
def join_chip_scores(rf, pdf, year, scene_grp):

    # Get tile geometries from RasterFrame
    geo_pdf = rf.withColumn('crs', rf_crs('Red')) \
                .withColumn('geometry', rf_geometry('Red')) \
                .select('tile_id', 's2_grid_id', 's2_id', 's2_datetime', 's2_eo_cloud_cover', 
                        'crs', 'geometry') \
                .toPandas()
    
    # Join with scores and create GeoDataFrame
    scores_pdf = pd.merge(geo_pdf, deployment_scores_pdf, 
                          how='inner', on=['tile_id', 's2_grid_id'])
    scores_gdf = gpd.GeoDataFrame(scores_pdf,
                                  geometry='geometry',
                                  crs=scores_pdf.crs[0].crsProj4) \
                    .drop('crs', axis=1)
    
    # Convert crs
    scores_gdf = scores_gdf.to_crs('EPSG:4326')
    
    # Update tile_id to include year and scene_grp
    scores_gdf.tile_id = [x+'-'+year+'-'+str(scene_grp).zfill(2) for x in scores_gdf.tile_id]
    
    # Return joined GeoDataFrame
    return(scores_gdf)

In [ ]:
def calc_inds_aggs(idf):
    
    # Includes area-weighted pred and max pred for cement and steel
    # Create list of indexes that intersect with chip
    odf = {}
    odf['tile_inds_cmt_pred_wavg'] = sum(idf.inds_cmt_pred*idf.area) / sum(idf.area)
    odf['tile_inds_stl_pred_wavg'] = sum(idf.inds_stl_pred*idf.area) / sum(idf.area)
    odf['tile_inds_cmt_pred_max'] = max(idf.inds_cmt_pred)
    odf['tile_inds_stl_pred_max'] = max(idf.inds_stl_pred)
    odf['tile_inds_ids'] = (',').join(str(x) for x in idf.inds_id.tolist())
    
    return pd.Series(odf, index=['tile_inds_ids', 'tile_inds_cmt_pred_wavg', 'tile_inds_stl_pred_wavg',
                                 'tile_inds_cmt_pred_max', 'tile_inds_stl_pred_max'])

In [ ]:
def merge_inds_info(base_gdf, inds_gdf):
    
    # Find intersection of base_gdf and inds_gdf and compute area
    # of the geometry
    insct_inds_gdf = gpd.overlay(base_gdf, inds_gdf, how='intersection')
    insct_inds_gdf = insct_inds_gdf.to_crs(calc_crs)
    insct_inds_gdf['area'] = insct_inds_gdf.area
    
    # Compute aggregations on inds_gdf by tile_id
    agg_inds_gdf = insct_inds_gdf.groupby('tile_id') \
                                 .apply(calc_inds_aggs) \
                                 .reset_index()
    
    # Join back to base_gdf and return
    output_gdf = pd.merge(base_gdf, agg_inds_gdf, how='left', on='tile_id')
    return(output_gdf)

In [ ]:
def merge_known_plants(base_gdf, plant_gdf):
    
    # uid column name (cmt or stl)
    uid_cname = plant_gdf.columns.tolist()[0]
    plnt_label = uid_cname.split('_')[0]+'_'
    
    # Spatial join base_gdf to plant_gdf to find where
    # plants are within tiles
    tile_gdf = base_gdf[['tile_id','geometry']]
    tile_gdf = gpd.sjoin(tile_gdf, plant_gdf, how='inner', op='intersects') \
                  .sort_values(uid_cname) \
                  .drop('index_right', axis=1) \
                  .reset_index(drop=True)
    
    # Proceed if find plants
    if len(tile_gdf) > 0:
        
        # Convert to physical crs
        tile_gdf['geometry'] = tile_gdf.geometry.centroid
        tile_gdf = tile_gdf.to_crs(calc_crs)
        
        # Get intersecting plants and convert to physical crs
        plant_phys_gdf = plant_gdf[plant_gdf[uid_cname].isin(tile_gdf[uid_cname].tolist())]
        plant_phys_gdf = plant_phys_gdf.reset_index(drop=True)
        plant_phys_gdf = plant_phys_gdf.to_crs(calc_crs)
        
        # Make sure id's all lined up
        vals_match = (tile_gdf[uid_cname] == plant_phys_gdf[uid_cname])
        if sum(vals_match) != len(vals_match):
            sys.exit('ERROR: Spatial join between tile and known plants is misaligned; cannot compute distance')
            
        # Calculate distance between chip center and plant
        tile_gdf['tile_'+plnt_label+'distm'] = tile_gdf.distance(plant_phys_gdf)
        
        # Join back to base_gdf
        merged_gdf = pd.merge(base_gdf, 
                              tile_gdf.drop('geometry', axis=1),
                              how='left', on='tile_id')
    
    else:
        
        # Add columns of NaNs
        merged_gdf = base_gdf
        merged_gdf[uid_cname] = np.nan
        merged_gdf['tile_'+plnt_label+'distm'] = np.nan
        
    return(merged_gdf)

In [ ]:
def write_chip_scores(output_gdf, year, scene_grp):

    # Write output to GeoJson
    output_score_file = output_path+'/'+output_gjson_prefix+output_gdf.s2_grid_id[0]+'-sg'+str(scene_grp).zfill(2)+'.geojson'
    output_gdf.to_file(output_score_file, driver='GeoJSON')
    
    # Return name of score file
    return(output_score_file)

## Read in Input Geometries for analysis

### Deployment Region and S2 scene list from 10km Grid

In [ ]:
macro_deployment_gdf = gpd.read_file(s2_deployment_gjson)
s2_scene_gdf = gpd.read_file(s2_scene_gjson)

### 10km Grid with Predictions from Infrastructure Density Model

In [ ]:
s2_grid_gdf = gpd.read_file(s2_grid_gjson)

#### Known cement and steel plants

In [ ]:
cmt_plant_gdf = gpd.read_file(cement_plant_gjson)
cmt_plant_gdf = cmt_plant_gdf[['uid', 'geometry']].rename(columns={'uid': 'cmtv4p1_uid'}) \
                                                  .sort_values('cmtv4p1_uid') \
                                                  .reset_index(drop=True)

In [ ]:
stl_plant_gdf = gpd.read_file(steel_plant_gjson)
stl_plant_gdf = stl_plant_gdf[['uid', 'geometry']].rename(columns={'uid': 'stlv4p1_uid'}) \
                                                  .sort_values('stlv4p1_uid') \
                                                  .reset_index(drop=True)

### Split scoring effort in four

In [ ]:
s2_scene_gdf['grid_int'] = [int(n.split('-')[1][0:2]) for n in s2_scene_gdf.grid_id.tolist()]
if scene_subset == 1:
    s2_scene_gdf = s2_scene_gdf[s2_scene_gdf.grid_int <= 48]
if scene_subset == 2:    
    s2_scene_gdf = s2_scene_gdf[(s2_scene_gdf.grid_int > 48) & (s2_scene_gdf.grid_id < 'MGRS-50QLK')]
if scene_subset == 3:
    s2_scene_gdf = s2_scene_gdf[s2_scene_gdf.grid_id >= 'MGRS-50QLK']
    s2_scene_gdf = s2_scene_gdf.iloc[0:200]
if scene_subset == 4:
    s2_scene_gdf = s2_scene_gdf[s2_scene_gdf.grid_id >= 'MGRS-50QLK']
    s2_scene_gdf = s2_scene_gdf.iloc[200:]
scene_ids = s2_scene_gdf.grid_id.tolist()

In [ ]:
print(len(scene_ids))

### Fail-safe

If server crashes, this picks up where we left off, so don't have to rerun scoring.

In [ ]:
scored_scene_list = os.listdir(output_path)
scored_scene_list.sort()
if len(scored_scene_list) > 0:
    last_scored_scene = ('-').join(scored_scene_list[-1].split('.')[0].split('-')[8:10])
    last_ind = scene_ids.index(last_scored_scene)
    # Repeat the last scene, in case didn't fully finish
    scene_ids = scene_ids[last_ind:]

In [ ]:
print(len(scene_ids))

### Temporary code to score specific scenes

## Loop over Scenes, Create Chips, and Score

For each scene:

* Get catalog of Sentinel-2 scenes between May and August of specified year that intersect with deployment region
* Read in scenes to create image chips
* Read in QA?
* Clip to deployment region
* Score models
* Write scores out to file

In [ ]:
# Delete temporary output paths for geotiffs and pngs and all files if they exist
if os.path.exists('geotiffs'):
    shutil.rmtree('geotiffs')
if os.path.exists('pngs'):
    shutil.rmtree('pngs')

In [ ]:
# Loop over all scenes to score:
for scene_id in scene_ids:
    
    # Time tracking:
    stime = time.time()
    
    # Clean up output
    clear_output(wait=True)
    
    # Feedback on progress
    print('Scoring Scene '+scene_id+' ('+str(scene_ids.index(scene_id)+1)+'/'+str(len(scene_ids))+')...')
    
    # Step 1: initilization of scene's deployment region
    #         (intersection of scene extent and deployment region)
    print('.....Computing intersection of scene with deployment region'+'...')
    scene_full_gdf = s2_scene_gdf[s2_scene_gdf.grid_id == scene_id]
    
    # Scale scene extent down a bit to avoid scoring the edges only
    scene_scaled_gdf = gpd.GeoDataFrame(geometry=scene_full_gdf.scale(scale_size, scale_size),
                                        crs='EPSG:4326')
    deployment_region_n_gdf = calc_scene_deployment_region(scene_scaled_gdf, 
                                                           macro_deployment_gdf)
    deployment_region_n_gdf = deployment_region_n_gdf[deployment_region_n_gdf.geometry.is_valid]
    
    # To keep track of number of chips scored for each scene
    tot_chp_scored = 0
    
    # Proceed if deployment region is not empty:
    if len(deployment_region_n_gdf) > 0:
    
        # Step 2: Find all scenes from May - August of specified year for specified
        #         scene_id; no limit on maximum cloud coverage
        print('.....Finding all scenes from May - August '+year+'...')
        site_cat_all = eod_read_catalog(deployment_region_n_gdf, scene_id, year, max_cc=100)
        scene_cnt = len(site_cat_all)
    
        # Proceed to next step if query returns results
        if scene_cnt > 0:
        
            # Loop over all returned scenes, in ascending order of cloud coverage
            scene_grp = list(range(0, scene_cnt))
            for n in scene_grp:
            
                # Feedback on progress
                print('.....Creating chips for group '+str(n+1)+'...')
    
                # Step 3: get scene with (n+1)th least cloud coverage
                #         and create chips
                site_cat = site_cat_all.iloc[[n]]
                site_cat = gpd.sjoin(deployment_region_n_gdf, site_cat)
                if len(site_cat) > 0:
                    site_chips = create_chips(site_cat, chip_size=chip_size)
                    isNotEmptyRF = len(site_chips.head(1))
                else:
                    isNotEmptyRF = 0
    
                # Proceed to next step if able to create chips
                if isNotEmptyRF == 1:
                
                    # Feedback on progress
                    print('.....Creating geotiffs and pngs for group '+str(n+1)+'...')
                
                    # Step 4: create pngs to score
                    png_from_rf(site_chips)
        
                    # Proceed to next step if pngs successfully created
                    if len(glob.glob('pngs/all/*.png')) > 0:
                    
                        # Feed back on progress
                        print('.....Scoring model for group '+str(n+1)+'...')
                    
                        # Step 5: score model and join to RasterFrame to create output
                        deployment_scores_pdf = score_pngs('pngs')
                        output_gdf = join_chip_scores(site_chips, deployment_scores_pdf, year, n+1)
                    
                        # Clean up temporary files
                        if os.path.exists('geotiffs'):
                            shutil.rmtree('geotiffs')
                        if os.path.exists('pngs'):
                            shutil.rmtree('pngs')
                        
                        # Step 6: merge output with results from infrastructure density model
                        print('.....Joining Sentinel-2 scores with infrastructure density model for group '+str(n+1)+'...')
                        output_gdf = merge_inds_info(output_gdf, s2_grid_gdf)
                        
                        # Step 7: merge output with known plants
                        print('.....Joining Sentinel-2 scores with known plants for group '+str(n+1)+'...')
                        output_gdf = merge_known_plants(output_gdf, cmt_plant_gdf)
                        output_gdf = merge_known_plants(output_gdf, stl_plant_gdf)
                    
                        # Step 8: write results out to GeoJSON
                        print('.....Writing scores to GeoJSON for group '+str(n+1)+'...')
                        oscore_file = write_chip_scores(output_gdf, year, n+1)
                        grp_chp_scored = len(output_gdf)
                        tot_chp_scored = tot_chp_scored + grp_chp_scored
                        print('.....Scored '+str(grp_chp_scored)+' chips in group '+str(n+1)+'...')
            
                        # Tidy up for next round
                        if os.path.exists('geotiffs'):
                            shutil.rmtree('geotiffs')
                        if os.path.exists('pngs'):
                            shutil.rmtree('pngs')
            
                        # Step 9: Compute remaining region left to score
                        print('.....Calculating remaining region to score...')
                        deployment_region_n_gdf = calc_region_difference(deployment_region_n_gdf,
                                                                     output_gdf)
                        deployment_region_n_gdf = deployment_region_n_gdf[deployment_region_n_gdf.geometry.is_valid]
                    
                        # Stop if no region left to score
                        if len(deployment_region_n_gdf) == 0:
                            print('.....Finished scoring scene '+scene_id+'.')
                            break
                        
                    # Feedback if no pngs created
                    else:
                        print('.....Unable to create pngs for group '+str(n+1)+': 0 tiles scored.')
            
                # Feedback if no chips created
                else:
                    print('.....Unable to create chips for group '+str(n+1)+': 0 tiles scored.')
        
        # Feedback if query returns no results
        else:
            print('.....EOD query did not return results for scene '+scene_id+': 0 tiles scored.')
    
    # Feedback is original deployment region is empty
    else:
        print('.....Scene '+scene_id+' does not intersect with deployment region: 0 tiles scored.')
        
    # Time tracking:
    etime = time.time()
    print('...Total number of chips scored for '+scene_id+': '+str(tot_chp_scored)+'.')
    print('...Total execution time for '+scene_id+': '+str((etime-stime)/60.)+' min.')

## Tar results and upload to S3

In [ ]:
unix_code = 'tar -cvf '+output_score_tar+' '+output_path

In [ ]:
os.system(unix_code)

In [ ]:
bucket.upload_file(output_score_tar, 
                   s3_path+'/'+output_score_tar)